In [296]:
import re

parts = []
workflows = {}

with open('data/19.txt', 'r') as file:
    data = file.read().splitlines()
    w, p = data[:data.index('')], data[data.index('') + 1:]

    for line in w:
        rule, conditions = re.findall(r'(\w+)\{(.*)\}', line)[0]
        conditions = [x.split(':') for x in conditions.split(',')]
        workflows[rule] = conditions

    for line in p:
        part = dict((p, int(v)) for p, v in re.findall(r'(\w)=(\d+)', line))
        parts.append(part)

In [297]:
def check(rule, part):
    match len(rule):
        case 1: return True
        case 2:
            (rule, _), key = rule, rule[0][0]
            return eval(rule.replace(key, str(part[key])))

def apply(workflow, part):
    for rule in workflows[workflow]:
        if (check(rule, part)):
            return rule[-1]

total = 0

for part in parts:
    next = apply('in', part)

    while not next in ['A', 'R']:
        next = apply(next, part)

    if next == 'A':
        total += sum(part.values())

total

492702

In [298]:
def new_part(part, key, values):
    part = part.copy()
    part[key] = values
    return part

def new_parts(current, rule, part):
    rule, next = rule
    key, sign, value = rule[0], rule[1], int(rule[2:])
    start, end = part[key]

    if sign == '>':
        s, e = max(value + 1, start), max(value - 1, end)
    else:
        s, e = min(value, start), min(value - 1, end)

    if s < e:
        # Move to the next workflow with matching range
        yield next, new_part(part, key, [s, e])

        # Check current workflow again with ranges that not match
        if start < s:
            yield current, new_part(part, key, [start, s - 1])
        
        if e < end:
            yield current, new_part(part, key, [e + 1, end])

def check(current, rule, part):
    match len(rule):
        case 1: return [(rule[0], part)]
        case 2: return [*new_parts(current, rule, part)]

def apply(workflow, part):
    for rule in workflows[workflow]:
        if result := check(workflow, rule, part):
            return result

In [299]:
import math

part = { 'x': [1, 4000],'m': [1, 4000],'a': [1, 4000],'s': [1, 4000]}

combinations = 0
queue = [('in', part)]

while queue:
    workflow, part = queue.pop()

    if workflow in 'AR':
        if workflow == 'A':
            combinations += math.prod(end - start + 1 for start, end in part.values())
        continue
    
    for w, p in apply(workflow, part):
        queue.append((w, p))

combinations

138616621185978